In [1]:
import os
import glob
import pandas as pd

In [2]:
pwd

'/Users/michaelpiacentino/Drive/git/data/dnBMPR1A/MigrationArea'

In [3]:
# Add analysis date here to apply to dataframe
analysis_date = '20181214'

path = os.path.abspath('')+'/CSVs/'
full_df = pd.DataFrame()
list_ = []

# For loop to bring in files and concatenate them into a single dataframe
for file_ in glob.glob(path + "/*.csv"):
    df = pd.read_csv(file_)
    # Determine Image name from file name
    df['Image'] = os.path.splitext(os.path.basename(file_))[0]
    # Split values in ROI label and Image name columns
    df['delete1'], df['ROI'], df['delete2'], df['delete3'] = zip(*df['Label'].map(lambda x: x.split(':')))
    (df['ExptDate'], df['Treatment'], df['Dose'], df['Stains'], df['Embryo'], 
        df['Somites'], df['ImageMag']) = zip(*df['Image'].map(lambda x: x.split('_')))
    list_.append(df)

full_df = pd.concat(list_)
full_df = full_df.drop(['Label', 'Mean', 'Area', 'RawIntDen', 'delete1', 'delete2', 'delete3'], axis=1)
full_df.head()

,,IntDen,Image,ROI,ExptDate,Treatment,Dose,Stains,Embryo,Somites,ImageMag
0,1,6.636673e+07,20181125_pCAGdnBMPR1AFLAG_3ugul_NC11m3eGFP_Emb...,CntlArea,20181125,pCAGdnBMPR1AFLAG,3ugul,NC11m3eGFP,Emb7,8ss,5x
1,2,4.417437e+07,20181125_pCAGdnBMPR1AFLAG_3ugul_NC11m3eGFP_Emb...,ExptArea,20181125,pCAGdnBMPR1AFLAG,3ugul,NC11m3eGFP,Emb7,8ss,5x
0,1,6.150788e+07,20181125_pCAGdnBMPR1AFLAG_3ugul_NC11m3eGFP_Emb...,CntlArea,20181125,pCAGdnBMPR1AFLAG,3ugul,NC11m3eGFP,Emb2,8ss,5x
1,2,3.879683e+07,20181125_pCAGdnBMPR1AFLAG_3ugul_NC11m3eGFP_Emb...,ExptArea,20181125,pCAGdnBMPR1AFLAG,3ugul,NC11m3eGFP,Emb2,8ss,5x
0,1,5.476126e+07,20181125_pCAGdnBMPR1AFLAG_3ugul_NC11m3eGFP_Emb...,CntlArea,20181125,pCAGdnBMPR1AFLAG,3ugul,NC11m3eGFP,Emb5,8ss,5x


In [7]:
# Get a list of treatments
treatment_list = full_df.Treatment.unique()
treatment_list = treatment_list.tolist()

# Group dataframe by treatment
df_byTreatment = full_df.groupby(['Treatment', 'Image', 'ROI'])['Image', 'ROI', 'IntDen', 'Treatment'].mean()

# Loop trough treatments, performing each analysis and exporting CSV file for each treatment
for i in treatment_list:
    # Slice dataframe to process only embryos with given treatment
    treatment = i
    df_treatment = pd.DataFrame(df_byTreatment.xs(treatment))

    # Extract Cntl and Expt IntDen
    intden_cntl = df_treatment.xs('CntlArea', level='ROI')
    intden_expt = df_treatment.xs('ExptArea', level='ROI')

    # Generate Expt/Cntl IntDen Ratios
    intden_ratios = pd.DataFrame(intden_expt / intden_cntl)
    intden_ratios.columns = ['Expt/Cntl IntDen']

    # Normalize all migration IntDen values to mean of control group
    norm_cntl = intden_cntl/(float(intden_cntl.mean()))
    norm_cntl.columns = ['Norm Cntl IntDen']
    norm_expt = intden_expt/(float(intden_cntl.mean()))
    norm_expt.columns = ['Norm Expt IntDen']
    norm_intden = pd.concat([norm_cntl, norm_expt], axis=1, sort=False)

    # Combine processed values into single dataframe and output as csv file analysis date + 'MigrationResults.csv'
    intden_cntl = pd.DataFrame(intden_cntl)
    intden_cntl.columns = ['Cntl IntDen']
    intden_expt = pd.DataFrame(intden_expt)
    intden_expt.columns = ['Expt IntDen']
    results = pd.concat([intden_cntl, intden_expt, intden_ratios, norm_cntl, norm_expt], axis=1, sort=True)
    results.to_csv(analysis_date + '_' + treatment + '_NC11m3IntDenResults.csv')